In [1]:
#Updated Model

In [3]:
# Updates to the model:
# 1. Adding two more hidden layers 
# 2. Running the model with a different number of features to see if accuracy and loss are affected
# 3. Altering activation functions to see if model accuracy improves 
# 4. Tested out different models such as a random forest model

In [4]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
import psycopg2

In [ ]:
# Postgres username, password, and database name (change accordingly)
postgres_address = 'localhost'
postgres_port = '5432' 
postgres_username = 'postgres' 
postgres_password = '********'
postgres_dbname = 'cardio_disease_db'
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=postgres_username,
                        password=postgres_password,
                        ipaddress=postgres_address,
                        port=postgres_port,
                        dbname=postgres_dbname))
# Create the connection
cnx = create_engine(postgres_str)

In [5]:
# Upload csv if cannot connect to database
# cardio_train_df = pd.read_csv('cardio_training_clean.csv')
# cardio_train_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
0,0,50,2,168,136.7,110,80,1,1,0,0,1,0,3.4
1,1,55,1,156,187.4,140,90,3,1,0,0,1,1,5.4
2,2,52,1,165,141.1,130,70,3,1,0,0,0,1,3.6
3,3,48,2,169,180.8,150,100,1,1,0,0,1,1,4.5
4,4,48,1,156,123.5,100,60,1,1,0,0,0,0,3.6


In [6]:
# Define feature data
# features: age, gender, height, weight, ap_hi, 
# ap_lo, cholesterol, gluc, smoke, alco, active 
X = cardio_train_df.drop(['id','cardio','smoke'],1).values

In [7]:
# Define target data
# cardio (if the person has cardiovascular disease or not) 
y = cardio_train_df['cardio']

In [8]:
# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

In [10]:
# Fit the StandardScaler
X_scaler.fit(X_train)

StandardScaler()

In [11]:
# Scale the features data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 6
hidden_nodes_layer5 = 6

nn_model = tf.keras.models.Sequential()

In [13]:
# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

In [14]:
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [15]:
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

In [16]:
# Fourth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

In [17]:
# Fifth hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

In [18]:
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [19]:
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 15)                195       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 128       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 498
Trainable params: 498
Non-trainable params: 0
__________________________________________________________

In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
1582/1582 [==============================] - 2s 903us/step - loss: 0.5959 - accuracy: 0.6892
Epoch 2/200
1582/1582 [==============================] - 1s 925us/step - loss: 0.5564 - accuracy: 0.7237
Epoch 3/200
1582/1582 [==============================] - 1s 845us/step - loss: 0.5514 - accuracy: 0.7268
Epoch 4/200
1582/1582 [==============================] - 1s 944us/step - loss: 0.5459 - accuracy: 0.7310
Epoch 5/200
1582/1582 [==============================] - 2s 954us/step - loss: 0.5493 - accuracy: 0.7288
Epoch 6/200
1582/1582 [==============================] - 2s 1ms/step - loss: 0.5487 - accuracy: 0.7288
Epoch 7/200
1582/1582 [==============================] - 1s 907us/step - loss: 0.5445 - accuracy: 0.7315
Epoch 8/200
1582/1582 [==============================] - 1s 875us/step - loss: 0.5475 - accuracy: 0.7304
Epoch 9/200
1582/1582 [==============================] - 1s 835us/step - loss: 0.5440 - accuracy: 0.73130s - loss: 0.5436 - accu
Epoch 10/200
1582/1582 [=========

1582/1582 [==============================] - 1s 788us/step - loss: 0.5386 - accuracy: 0.7369
Epoch 154/200
1582/1582 [==============================] - 1s 789us/step - loss: 0.5369 - accuracy: 0.7399
Epoch 155/200
1582/1582 [==============================] - 1s 799us/step - loss: 0.5386 - accuracy: 0.7370
Epoch 156/200
1582/1582 [==============================] - 1s 792us/step - loss: 0.5414 - accuracy: 0.7356
Epoch 157/200
1582/1582 [==============================] - 1s 791us/step - loss: 0.5436 - accuracy: 0.7332
Epoch 158/200
1582/1582 [==============================] - 1s 803us/step - loss: 0.5392 - accuracy: 0.7355
Epoch 159/200
1582/1582 [==============================] - 1s 806us/step - loss: 0.5405 - accuracy: 0.7355
Epoch 160/200
1582/1582 [==============================] - 1s 785us/step - loss: 0.5414 - accuracy: 0.7337
Epoch 161/200
1582/1582 [==============================] - 1s 858us/step - loss: 0.5390 - accuracy: 0.7352
Epoch 162/200
1582/1582 [==========================

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

528/528 - 0s - loss: 0.5458 - accuracy: 0.7307
Loss: 0.5458288192749023, Accuracy: 0.7307213544845581


In [ ]:
# Close the cursor and connection so the server can allocate 
#bandwidth to other requests
cur.close()
conn.close()